In [18]:
import pandas as pd
import numpy as np

In [19]:
train_df = pd.read_csv('../data/train_dataV2.csv')
val_df = pd.read_csv('../data/validation_dataV2.csv')
test_df = pd.read_csv('../data/test_dataV2.csv')

In [20]:
train_df.head()

,CategoryCode,ItemCode,DateID,DailySales,DOW,WOY,DOY
0,category_2,117610,2021-11-06,7,5,44,310
1,category_4,836584,2021-11-18,16,3,46,322
2,category_1,370195,2022-01-24,6,0,4,24
3,category_2,172582,2021-10-30,5,5,43,303
4,category_2,1006009,2021-10-30,5,5,43,303


In [21]:
train_udf = train_df.copy().iloc[:, 1:4]
train_udf.head()

,ItemCode,DateID,DailySales
0,117610,2021-11-06,7
1,836584,2021-11-18,16
2,370195,2022-01-24,6
3,172582,2021-10-30,5
4,1006009,2021-10-30,5


In [22]:
train_udf.shape

(19921, 3)

In [23]:
start_date = min(train_udf.DateID)
end_date = max(train_udf.DateID)
print(start_date, end_date)
def df_func(x):
    _x = x.copy().reset_index(drop=True)
    _x.drop(columns=['ItemCode'], inplace=True)
    _x['DateID'] = pd.to_datetime(_x['DateID'])
    _x = _x.set_index('DateID')
    __x = _x.resample('D').sum().replace(0, np.NaN)
    return __x

2021-10-01 2022-02-13


In [24]:
res_df = train_udf.groupby('ItemCode').apply(lambda x: df_func(x)).reset_index()
# train_udf.groupby('ItemCode').apply(lambda x: print(x.index)).reset_index()
res_df.head()

,ItemCode,DateID,DailySales
0,3418,2021-10-01,18.0
1,3418,2021-10-02,3.0
2,3418,2021-10-03,8.0
3,3418,2021-10-04,2.0
4,3418,2021-10-05,4.0


In [25]:
res_df.shape

(25946, 3)

In [26]:
res_df.isnull().sum()

ItemCode         0
DateID           0
DailySales    6025
dtype: int64

In [27]:
I2C_map = pd.Series(train_df.CategoryCode.values,index=train_df.ItemCode).to_dict()
res_df['CategoryCode'] = res_df['ItemCode'].map(I2C_map)

In [28]:
res_df.columns = ['ItemCode', 'Date', 'DailySales', 'CategoryCode']
res_df.head()

,ItemCode,Date,DailySales,CategoryCode
0,3418,2021-10-01,18.0,category_1
1,3418,2021-10-02,3.0,category_1
2,3418,2021-10-03,8.0,category_1
3,3418,2021-10-04,2.0,category_1
4,3418,2021-10-05,4.0,category_1


In [30]:
res_df.to_csv('../data/train_dataV6.csv', index=False)

---

In [91]:
train_items = set(train_df['ItemCode'])
val_items = set(val_df['ItemCode'])
test_items = set(test_df['ItemCode'])

In [92]:
stage_map = {i: 'validation' for i in val_items}
stage_map.update({i: 'test' for i in test_items})
stage_map.update({836584: 'train-only', 1090249: 'train-only'})

In [113]:
df2 = res_df.copy()
df2.head()

,ItemCode,Date,DailySales,CategoryCode
0,3418,2021-10-01,18,category_1
1,3418,2021-10-02,3,category_1
2,3418,2021-10-03,8,category_1
3,3418,2021-10-04,2,category_1
4,3418,2021-10-05,4,category_1


In [135]:
sales_point = df2.groupby('ItemCode')['DailySales'].count().reset_index()
sales = df2.groupby('ItemCode')['DailySales'].agg(list)
sales_point['stage'] = sales_point['ItemCode'].map(stage_map)
sales_point['sales'] = sales_point['ItemCode'].map(sales)
sales_point.columns = ['ItemCode', '#sales', 'stage', 'DailySales']
sales_point.head()
# sales_point1.to_csv('../data/SalesPointsV1.csv')

,ItemCode,#sales,stage,DailySales
0,3418,136,validation,"[18, 3, 8, 2, 4, 2, 8, 12, 0, 14, 2, 9, 8, 5, ..."
1,3427,136,validation,"[2, 7, 2, 3, 9, 0, 5, 8, 10, 5, 5, 4, 0, 4, 3,..."
2,7666,135,validation,"[14, 58, 12, 8, 5, 33, 11, 23, 24, 15, 41, 20,..."
3,9925,133,test,"[1, 3, 1, 2, 0, 0, 2, 1, 0, 1, 8, 0, 2, 0, 0, ..."
4,16936,136,test,"[4, 1, 4, 1, 0, 0, 21, 2, 1, 0, 5, 9, 2, 8, 8,..."


In [114]:
df2['WoY'] = df2["Date"].dt.weekofyear
df2.head()

C:\Users\deela\AppData\Local\Temp/ipykernel_34440/2620650257.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df2['WoY'] = df2["Date"].dt.weekofyear


,ItemCode,Date,DailySales,CategoryCode,WoY
0,3418,2021-10-01,18,category_1,39
1,3418,2021-10-02,3,category_1,39
2,3418,2021-10-03,8,category_1,39
3,3418,2021-10-04,2,category_1,40
4,3418,2021-10-05,4,category_1,40


In [123]:
df2['WoYY'] = df2.apply(lambda x: f"{str(x.Date).split('-')[0]}-{x.WoY}", axis=1)
# df2.apply(lambda x: print(x['Date'], x.WoY), axis=1)
df2.head()

,ItemCode,Date,DailySales,CategoryCode,WoY,WoYY
0,3418,2021-10-01,18,category_1,39,2021-39
1,3418,2021-10-02,3,category_1,39,2021-39
2,3418,2021-10-03,8,category_1,39,2021-39
3,3418,2021-10-04,2,category_1,40,2021-40
4,3418,2021-10-05,4,category_1,40,2021-40


In [124]:
df3 = df2.loc[:, ['ItemCode', 'DailySales', 'WoYY']]
df3.head()

,ItemCode,DailySales,WoYY
0,3418,18,2021-39
1,3418,3,2021-39
2,3418,8,2021-39
3,3418,2,2021-40
4,3418,4,2021-40


In [127]:
def df_func1(x):
    _x = x.copy().reset_index(drop=True)
    _x.drop(columns=['ItemCode'], inplace=True)
    print(_x)
    __x = _x.groupby('WoYY').sum().reset_index()
    print(__x)
    return __x

In [130]:
df5 = df3.groupby(['ItemCode', 'WoYY']).sum().reset_index()
df5.to_csv('../data/train_dataV5.csv')

In [137]:
sales_point1 = df5.groupby('ItemCode')['DailySales'].count().reset_index()
sales = df5.groupby('ItemCode')['DailySales'].agg(list)
sales_point1['stage'] = sales_point1['ItemCode'].map(stage_map)
sales_point1['sales'] = sales_point1['ItemCode'].map(sales)
sales_point1.columns = ['ItemCode', '#sales', 'stage', 'WeeklySales']
sales_point1.head()
sales_point1.to_csv('../data/SalesPointsV2.csv')

In [ ]:
df4 = df3.groupby('ItemCode').apply(lambda x: df_func1(x))
df4.head()

In [100]:
df4.to_csv('../data/train_dataV4.csv')